In [ ]:
import preamble
from robot import Robot
from rgbd_stream import RGBDStream_iOS
from calibrator import Calibrator
from virtual_dynamics import SimpleVirtualDynamics
from timer import Timer
import numpy as np

In [ ]:
robot = Robot("169.254.9.43",
              translational_force_deadband=3.0,
              rotational_force_deadband=0.5)
robot.control.zeroFtSensor()

In [ ]:
stream = RGBDStream_iOS()
calibrator = Calibrator(stream, robot,
                        marker_color=np.array([0.13, 0.65, 0.6]), N=500)
dynamics = SimpleVirtualDynamics(M=np.array([20.0, 20.0, 20.0, 1.0, 1.0, 1.0]),
                                 B=np.array([50.0, 50.0, 50.0, 8.0, 8.0, 8.0]),
                                 K=0.0)
timer = Timer()

while calibrator.is_calibrating():
    dt = timer.dt()

    force = robot.get_force(Robot.TRANSLATION_ROTATION)
    dynamics.apply_force(force, dt)

    robot.set_velocity(dynamics.get_velocity(),
                       Robot.TRANSLATION_ROTATION, acceleration=150)

    calibrator.calibrate()

robot.set_velocity(Robot.zeroed_translation_rotation())
matrix = calibrator.compute_calibration_matrix()
np.save('calibration_matrix.npy', matrix)